In [1]:
import json
import requests

In [2]:
with open("tokens.json") as f:
    tokens = json.load(f)
    print(tokens.keys())

dict_keys(['consumer_key', 'consumer_secret', 'access_toke', 'access_token_secret', 'bearer_token'])


### Api reference
https://developer.twitter.com/en/docs/twitter-api/tweets/search/api-reference

- `GET /2/tweets/search/recent`
- `GET /2/tweets/search/all` (Academic Research application is required)

### How to build a query
For API v.2 we use `Core` operators only
- https://developer.twitter.com/en/docs/twitter-api/tweets/search/integrate/build-a-query#availability
- https://developer.twitter.com/en/docs/twitter-api/tweets/search/integrate/build-a-query#list



In [3]:
search_tweets_url = "https://api.twitter.com/2/tweets/search/recent"

headers = {
    "Authorization": "Bearer " + tokens["bearer_token"]
}

cian_params = {
    "query" : "циан lang:ru -from:cian_realty",
    # "sort_order": "relevancy",
    # "end_time": "2022-02-18T23:59:59Z",
    "max_results": 10
}

In [4]:
response = requests.get(
    search_tweets_url,
    headers=headers,
    params=cian_params
)

In [6]:
response

<Response [200]>

In [8]:
response.json()

{'data': [{'id': '1495061251927515137',
   'text': 'RT @antisekisov: зашел на циан прицениться к хатам и раскрыл убийство https://t.co/DeVOP8AX14'},
  {'id': '1495054177218617347',
   'text': 'мои любимые соц сети это хэдхантер и циан'},
  {'id': '1495040060701483010',
   'text': 'Главная идея вот этих приложений вроде Авито, Циан, Лалаво или Яндекс.Недвижимость в том, чтобы не было лишних физлиц или юрлиц между арендодателями и арендаторов, чтобы не было лишних трат на найм риелторов ЖЕ? Но в этих приложениях одни агенты и риелторы. Или я тупой?'},
  {'id': '1495026586852241410',
   'text': '@dtroode @cptnsir тебя убьёт циан где ты не сможешь снять хату а меня ёбнет ариец златовласый'},
  {'id': '1495007535358693380',
   'text': 'у меня есть два настроения: циан и озон'},
  {'id': '1494993578816327681',
   'text': 'проснулась сегодня в 8.30 и решила «просто полистать» циан\n\nк 11.30 уже получила одобрение ипотеки\nк 15.00 назначила встречу с риэлтором на завтра\nк 16.00 договорилась 

In [9]:
texts = [x["text"] for x in response.json()["data"]]

In [11]:
len(texts)

10

### Primitive preprocessing

In [12]:
from collections import Counter
from nltk.tokenize import WhitespaceTokenizer, WordPunctTokenizer, TreebankWordTokenizer
from nltk.corpus import stopwords

In [19]:
tokenizer = TreebankWordTokenizer()
listoftokens = [tokenizer.tokenize(x.lower()) for x in texts]
tokens = sum(listoftokens, [])
words_cnt = Counter(tokens)
words_cnt.most_common(20)

[('циан', 10),
 ('и', 8),
 (':', 7),
 (',', 7),
 ('на', 6),
 ('@', 5),
 ('к', 5),
 ('в', 4),
 ('не', 4),
 ('https', 3),
 ('или', 3),
 ('?', 3),
 ('rt', 2),
 ('зашел', 2),
 ('прицениться', 2),
 ('хатам', 2),
 ('раскрыл', 2),
 ('убийство', 2),
 ('//t.co/devop8ax14', 2),
 ('это', 2)]

In [21]:
stopwords.words('russian')[:5]

['и', 'в', 'во', 'не', 'что']

In [27]:
import string 
punct = string.punctuation

In [30]:
russian_stopwords = stopwords.words('russian')
russian_stopwords.extend([x for x in punct])
russian_stopwords = set(russian_stopwords)

words_cnt = Counter([token for token in tokens if token not in russian_stopwords])
words_cnt.most_common(20)

[('циан', 10),
 ('https', 3),
 ('rt', 2),
 ('зашел', 2),
 ('прицениться', 2),
 ('хатам', 2),
 ('раскрыл', 2),
 ('убийство', 2),
 ('//t.co/devop8ax14', 2),
 ('это', 2),
 ('этих', 2),
 ('авито', 2),
 ('лишних', 2),
 ('квартиры', 2),
 ('antisekisov', 1),
 ('мои', 1),
 ('любимые', 1),
 ('соц', 1),
 ('сети', 1),
 ('хэдхантер', 1)]